In [ ]:
!python3 -m pip install nltk
!python3 -m nltk.downloader stopwords
!unzip Dataset.zip

In [ ]:
import cudf
from cuml.preprocessing.model_selection import train_test_split
from cuml.experimental.preprocessing import StandardScaler
#from sklearn.feature_extraction.text import CountVectorizer
from cuml.feature_extraction.text import CountVectorizer

In [ ]:
label_list=[]
text_list=[]
file_list=[]
fileDir="Section1-Chapter3"


df = cudf.read_csv(fileDir+"/Labels.csv")
for row in df.to_pandas().itertuples(index=True, name='Pandas'):
    fileName=row.FileName+'.asm'
    with open("Section1-Chapter3/"+fileName, encoding='utf-8', errors='ignore') as f:
        content=f.read()
    content.replace(",", " ")
    content.replace('"', " ")
    text_list.append(content)
    file_list.append(fileName)
    label_list.append(row.Label)   

df['Text']=cudf.Series(text_list)


In [ ]:
count_vectorizer=CountVectorizer(stop_words='english',max_features=100)
features = count_vectorizer.fit_transform(df['Text'].to_pandas())

In [ ]:
df_count = cudf.DataFrame(features.toarray(), columns=count_vectorizer.get_feature_names())
df_count['FileName']=cudf.Series(file_list)
df_count['Label']=cudf.Series(label_list)

Label_CV=df_count['Label']
Features_CV = df_count.drop( columns=['FileName', 'Label'], axis=1)

X_CF_Normalized=StandardScaler().fit_transform(Features_CV)
X_CF_Train, X_CF_Test, Y_CF_Train, Y_CF_Test= train_test_split(X_CF_Normalized,Label_CV,test_size=0.20, shuffle=True)

In [ ]:
from cuml.linear_model import LogisticRegression

logReg_cf=LogisticRegression()
logReg_cf.fit(X_CF_Train,Y_CF_Train)
logReg_cf.score(X_CF_Train,Y_CF_Train)

In [ ]:
from cuml.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=10)

knn.fit(X_CF_Train,Y_CF_Train)
knn.score(X_CF_Train,Y_CF_Train)